In [1]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile


from bs4 import BeautifulSoup
import jinja2
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license, get_text
from tessa_chef import get_section_filename, create_predictable_zip, download_file
from tessa_chef import shutil



from le_utils.constants import content_kinds, file_formats, licenses

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)


## Test download_module funcion


In [2]:
# from tessa_chef import download_module
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108' # EN
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82912' # AR
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR

# toc = download_module(test_module)


In [3]:
# prints TOC
# for section in toc['children']:
#     print('  - section:', section['title'], section['filename'])
#     for subsection in section['children']:
#         print('    - subsection:', subsection['title'], subsection['filename'])




## Test scraping


In [3]:
from tessa_chef import TessaChef
tessa_chef = TessaChef()
tessa_chef.scrape(None, {'lang':'fr'})

Scrapring module @ url = http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=105334
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpdrsqxcwf


AttributeError: 'NoneType' object has no attribute 'find_parent'

In [4]:
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR
download_module_no_toc(test_module, lang='fr')

Scrapring module @ url = http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmp02cntxax
{'kind': 'TessaModuleContentsDict', 'lang': 'fr', 'source_id': '82514', 'title': 'Module 1 : Étude du nombre et de la structure', 'children': []}



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514
Scrapring section/subsectino... section-1.html
  - section: Module 1 : Étude du nombre et de la structure



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=1.1
Scrapring section/subsectino... section-1_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Utilisation de jeux pour développer le calcul mental



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=1.1
Scrapring section/subsectino... section-1_1.html
    - subsection: Module 1 : Étude du nombre et de la st




processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=3
Scrapring section/subsectino... section-3.html
  - section: Module 1 : Étude du nombre et de la structure: Section numéro 3 : Comment résoudre les problèmes numériques



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=3
Scrapring section/subsectino... section-3.html
  - section: Module 1 : Étude du nombre et de la structure: Section numéro 3 : Comment résoudre les problèmes numériques



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=3.1
Scrapring section/subsectino... section-3_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Il existe plusieurs solutions à un même problème : aider les élèves à réfléchir et à verbaliser leurs réponses



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=3.1
Scrapring sectio

  - section: Module 1 : Étude du nombre et de la structure: Section numéro 5 : Travail pratique avec les fractions



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=5.1
Scrapring section/subsectino... section-5_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Explorer les fractions simples en groupes, avec des ressources simples



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=5.1
Scrapring section/subsectino... section-5_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Explorer les fractions simples en groupes, avec des ressources simples



processing current_url http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514&section=5.2
Scrapring section/subsectino... section-5_2.html
    - subsection: Module 1 : Étude du nombre et de la structure: 2. Utiliser des bandes et des cercles de fractions pour faire des additions e

'/var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmp4ls586ao.zip'

In [6]:
from tessa_chef import urlparse, parse_qs, get_parsed_html_from_url
from tessa_chef import _get_next_section_url, download_page

def download_module_no_toc(module_url, lang=None):
    """
    Extracting the module table of contents from the sidebad nav doesn't work for certain modules in FR
    e.g. http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=105334&section=1.1
    
    If NO TOC is available, then we'll crawl pages one by one
    (`module_contents_dict`)
    """
    print('Scrapring module @ url =', module_url)
    doc = get_parsed_html_from_url(module_url)
    destination = tempfile.mkdtemp()
    print('destination=', destination)

    # copy css/js/images from skel
    shutil.copytree('chefdata/templates/module_skel/styles', os.path.join(destination,'styles'))

    source_id = parse_qs(urlparse(module_url).query)['id'][0]
    raw_title = doc.select_one("head title").text
    module_title = raw_title.replace('OLCreate:', '')\
            .replace('TESSA_ARABIC', '')\
            .replace('TESSA_Eng', '')\
            .replace('TESSA_Fr', '')\
            .strip()

    module_contents_dict = dict(
        kind='TessaModuleContentsDict',
        lang=lang,
        source_id=source_id,
        title=module_title,
        children=[],
    )
    print(module_contents_dict)
    
    
    
    

    # recusively download all sections by following "Next" links
    current_url = module_url
    current_section = None
    is_first_section = True
    while True:
        print('\n\n')
        print('processing current_url', current_url)
        current_doc = get_parsed_html_from_url(current_url)

        
        # special handling for module-level page (no section in url but is really Section 1)
        if is_first_section:
            section_filename = 'section-1.html'
            is_first_section = False
        else:
            section_filename = get_section_filename(current_url)

            
        # Do the actual download
        download_page(current_url, destination, section_filename, module_contents_dict)
        

        
        # Store section/subsecito info so we can build TOC later
        doc = get_parsed_html_from_url(current_url)
        raw_title = doc.select_one("head title").text
        the_title = raw_title.replace('OLCreate:', '')\
                .replace('TESSA_ARABIC', '')\
                .replace('TESSA_Eng', '')\
                .replace('TESSA_Fr', '')\
                .strip()

        # sections e.g. section-3.html
        if '_' not in section_filename:
            section_dict = dict(
                kind='TessaModuleSection',
                title=the_title,
                href=current_url,
                filename=section_filename,
                children=[]
            )
            module_contents_dict['children'].append(section_dict)
            print('  - section:', the_title)
            current_section = section_dict

        # subsections e.g. section-3_2.html
        else:
            subsection_dict = dict(
                kind='TessaModuleSubsection',
                title=the_title,
                href=current_url,
                filename=section_filename,
            )
            print('    - subsection:', the_title)
            current_section['children'].append(subsection_dict)

        
        # Recurse if next
        next_url = _get_next_section_url(current_doc)
        if next_url:
            current_url = next_url
        else:
            break

    # for debugging...
    pp.pprint(module_contents_dict)


    module_index_tmpl = jinja2.Template(open('chefdata/templates/module_index.html').read())
    index_contents = module_index_tmpl.render(module=module_contents_dict)
    with open(os.path.join(destination, "index.html"), "w") as f:
        f.write(index_contents)

    # return module_contents_dict
    zip_path = create_predictable_zip(destination)
    return zip_path

